# Advent of Code

In [1]:
import pandas as pd
import numpy as np

## 1a

In [2]:
s = pd.read_csv('data/day1.txt', header=None, squeeze=True)
s.diff().gt(0).sum()

1374

## 1b

In [3]:
s.rolling(3).sum().diff().gt(0).sum()

1418

## 2a

In [4]:
df = pd.read_csv('data/day2.txt', header=None, sep=' ', names=['direction', 'amount'])
df.head()

,direction,amount
0,forward,3
1,down,7
2,forward,8
3,down,9
4,forward,3


In [5]:
s = df.groupby('direction')['amount'].sum()
s

direction
down       2205
forward    1832
up         1033
Name: amount, dtype: int64

In [6]:
s['forward'] * (s['down'] - s['up'])

2147104

## 2b

In [7]:
# numpy
is_forward = df['direction'] == 'forward'
is_up = df['direction'] == 'up'
aim = np.where(is_forward, 0, 
         np.where(is_up, df['amount'] * -1, df['amount'])).cumsum()

depth = np.where(is_forward, aim * df['amount'], 0).sum()
horiz = df.loc[is_forward, 'amount'].sum()
depth * horiz

2044620088

In [8]:
# pandas
df['aim'] = (df['amount'] * (df['direction'] == 'down') - 
             df['amount'] * (df['direction'] == 'up')).cumsum()
df['depth'] = df['aim'] * df['amount'] * (df['direction'] == 'forward')
df

,direction,amount,aim,depth
0,forward,3,0,0
1,down,7,7,0
2,forward,8,7,56
3,down,9,16,0
4,forward,3,16,48
...,...,...,...,...
995,down,9,1155,0
996,down,2,1157,0
997,down,6,1163,0
998,down,9,1172,0


In [9]:
(df['amount'] * (df['direction'] == 'forward')).sum() * df['depth'].sum()

2044620088

## 3a

In [10]:
s = pd.read_csv('data/day3.txt', header=None, squeeze=True, dtype='str')
df = s.str.split("", expand=True).iloc[:, 1:-1]
df.head()

,1,2,3,4,5,6,7,8,9,10,11,12
0,0,1,1,1,0,1,1,0,1,1,1,0
1,0,1,0,1,1,0,0,0,1,1,0,1
2,1,0,0,1,1,1,0,0,0,1,1,0
3,0,1,1,1,1,0,1,0,1,0,0,0
4,1,0,1,1,0,1,0,0,0,1,0,0


In [11]:
s1 = df.mode().squeeze()
s1

1     0
2     1
3     1
4     1
5     0
6     0
7     1
8     0
9     1
10    1
11    0
12    0
Name: 0, dtype: object

In [12]:
a = int(s1.sum(), base=2)
a

1836

In [13]:
b = int((1 - s1.astype('int')).astype('str').sum(), base=2)
a * b

4147524

## 3b

In [14]:
df = s.str.split('', expand=True).iloc[:, 1:-1].astype('int')
df.head()

,1,2,3,4,5,6,7,8,9,10,11,12
0,0,1,1,1,0,1,1,0,1,1,1,0
1,0,1,0,1,1,0,0,0,1,1,0,1
2,1,0,0,1,1,1,0,0,0,1,1,0
3,0,1,1,1,1,0,1,0,1,0,0,0
4,1,0,1,1,0,1,0,0,0,1,0,0


In [15]:
# one solution
final = 1
for small in (False, True):
    df1 = df
    for col in df1.columns:
        if len(df1) == 1:
            continue
        mode = df1[col].mode()
        if len(mode) == 1:
            mode = mode.iloc[0]
        else:
            mode = 1
        if small:
            mode = 1 - mode
        df1 = df1[df1[col] == mode]
    final *= int(df1.squeeze().astype('str').sum(), base=2)
final

3570354

In [16]:
# another solution
final = 1
for asc in (True, False):
    df1 = df
    for i in range(1, 13):
        vc = df1[i].value_counts().reset_index().sort_values([i, 'index'], ascending=asc)
        val = vc.iloc[0, 0]
        df1 = df1[df1[i] == val]
    final *= int(df1.squeeze().astype('str').sum(), base=2)
final

3570354

## 4a

In [17]:
s = pd.read_csv('data/day4.txt', nrows=1,  header=None).squeeze()
s.head()

0    23
1    30
2    70
3    61
4    79
Name: 0, dtype: int64

In [18]:
df = pd.read_csv('data/day4.txt', skiprows=[0, 1], header=None, skip_blank_lines=True, sep='\s+')
df = df.set_index(df.index // 5)
df

,0,1,2,3,4
0,50,98,65,14,47
0,0,22,3,83,46
0,87,93,81,84,58
0,40,35,28,74,48
0,45,99,59,37,64
...,...,...,...,...,...
99,88,62,76,78,95
99,64,65,36,58,22
99,7,21,98,93,42
99,79,99,9,89,10


In [19]:
df1 = df
for num in s:
    df1 = df1.mask(df1 == num)
    is_bingo = df1.isna()
    horiz = is_bingo.all(axis=1)[lambda x: x]
    vert = is_bingo.groupby(level=0).all().any(axis=1)[lambda x: x]
    if len(horiz):
        winner = horiz.index[0]
        break
    if len(vert):
        winner = vert.index[0]
        break
df1.loc[winner].stack().sum() * num    

31424.0

## 4b

In [20]:
%%time
df1 = df
for num in s:
    df1 = df1.mask(df1 == num)
    is_bingo = df1.isna()
    horiz = is_bingo.all(axis=1)[lambda x: x]
    vert = is_bingo.groupby(level=0).all().any(axis=1)[lambda x: x]
    winners = horiz.index.union(vert.index)
    if len(df1) == 5:
        total = df1.stack().sum() * num  
    df1 = df1[~df1.index.isin(winners)]
    if len(df1) == 0:
        break
total

CPU times: user 87.2 ms, sys: 2.61 ms, total: 89.8 ms
Wall time: 88.7 ms


23042.0

## 5a

In [21]:
df = pd.read_csv('data/day5.txt', sep='\D+', 
            header=None, engine='python',
           names=['x1', 'y1', 'x2', 'y2'])
df.head()

,x1,y1,x2,y2
0,599,531,599,32
1,435,904,435,489
2,768,714,768,187
3,845,552,596,801
4,167,680,167,445


In [22]:
dfs = []
for a, b in ('xy', 'yx'):
    df_temp = (df.query(f'{a}1 == {a}2')
                 .set_index(f'{a}1')[[f'{b}1', f'{b}2']]
                 .apply(lambda x: range(min(x), max(x) + 1), axis=1)
                 .explode()
                 .reset_index(name=f'{b}1'))
    dfs.append(df_temp)
pd.concat(dfs).value_counts().gt(1).sum()

7644

## 5b

In [23]:
df1 = df.query('abs(x1 - x2) == abs(y1 - y2)')
df1.head()

,x1,y1,x2,y2
3,845,552,596,801
6,780,295,179,896
7,310,539,602,831
12,689,815,73,199
16,315,532,773,74


In [24]:
dfs = []
for a, b in ('xy', 'yx'):
    df_temp = (df.query(f'{a}1 == {a}2')
                 .set_index(f'{a}1')[[f'{b}1', f'{b}2']]
                 .apply(lambda x: range(min(x), max(x) + 1), axis=1)
                 .explode()
                 .reset_index(name=f'{b}1'))
    dfs.append(df_temp)

x1 = df1.apply(lambda x: np.linspace(x['x1'], x['x2'], abs(x['x2'] - x['x1']) + 1), axis=1).explode()
y1 = df1.apply(lambda x: np.linspace(x['y1'], x['y2'], abs(x['y2'] - x['y1']) + 1), axis=1).explode()
df_diag = pd.DataFrame({'x1': x1, 'y1': y1}).astype('int')
dfs.append(df_diag)

In [25]:
pd.concat(dfs).value_counts().gt(1).sum()

18627